# Test des Corrections du Niveau 0

**Objectif :** Valider que les corrections apportées ont résolu les problèmes :
- Réduction du taux de rejet excessif (61.6% → ~10-15%)
- Correction de l'algorithme de répétitions
- Calibrage automatique plus permissif

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import time

# Configuration paths
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Import du module corrigé
from detection.level0_prefilter import QualityFilter, auto_calibrate_filter

print("✅ Module corrigé importé")

In [ ]:
# Charger les données
data_path = os.path.join(project_root, 'data', 'results', 'batch_summary_production.csv')
df = pd.read_csv(data_path)

summaries_data = []
for idx, row in df.iterrows():
    summaries_data.append({
        'id': f"{row['text_id']}_{row['fusion_strategy']}",
        'text': row['summary'],
        'quality_grade': row['quality_grade']
    })

print(f"📊 {len(summaries_data)} résumés chargés")

In [ ]:
# Test rapide avec seuils manuels corrigés
print("🔧 Test avec seuils manuels corrigés...")
filter_corrected = QualityFilter()

start_time = time.time()
valid_corrected, results_corrected = filter_corrected.process_batch(summaries_data)
time_corrected = time.time() - start_time

stats_corrected = filter_corrected.get_statistics(results_corrected)

print(f"\n📈 Résultats après corrections:")
print(f"   Seuils: {filter_corrected.min_words}-{filter_corrected.max_words} mots")
print(f"   Résumés valides: {len(valid_corrected)}/{len(summaries_data)} ({len(valid_corrected)/len(summaries_data)*100:.1f}%)")
print(f"   Taux rejet: {stats_corrected['rejection_rate_percent']:.1f}%")
print(f"   Temps moyen: {stats_corrected['avg_processing_time_ms']:.1f}ms")

print(f"\n❌ Principales raisons de rejet:")
for reason, count in list(stats_corrected['rejection_reasons'].items())[:5]:
    print(f"   {reason}: {count} cas ({count/len(summaries_data)*100:.1f}%)")

In [ ]:
# Test calibrage automatique corrigé
print("🎯 Test calibrage automatique corrigé...")
filter_auto_corrected = auto_calibrate_filter(summaries_data)

start_time = time.time()
valid_auto_corrected, results_auto_corrected = filter_auto_corrected.process_batch(summaries_data)
time_auto_corrected = time.time() - start_time

stats_auto_corrected = filter_auto_corrected.get_statistics(results_auto_corrected)

print(f"\n📈 Résultats calibrage automatique corrigé:")
print(f"   Seuils auto: {filter_auto_corrected.min_words}-{filter_auto_corrected.max_words} mots")
print(f"   Résumés valides: {len(valid_auto_corrected)}/{len(summaries_data)} ({len(valid_auto_corrected)/len(summaries_data)*100:.1f}%)")
print(f"   Taux rejet: {stats_auto_corrected['rejection_rate_percent']:.1f}%")
print(f"   Temps moyen: {stats_auto_corrected['avg_processing_time_ms']:.1f}ms")

In [ ]:
# Comparaison avant/après corrections
print("📊 COMPARAISON AVANT/APRÈS CORRECTIONS:")
print("\n🔴 AVANT (problématique):")
print("   Manuel: 143/372 valides (38.4%) - Taux rejet: 61.6%")
print("   Auto: 99/372 valides (26.6%) - Taux rejet: 73.4%")
print("   Problème: 15,884 cas de répétitions détectées (impossible)")

print("\n🟢 APRÈS (corrigé):")
print(f"   Manuel: {len(valid_corrected)}/{len(summaries_data)} valides ({len(valid_corrected)/len(summaries_data)*100:.1f}%) - Taux rejet: {stats_corrected['rejection_rate_percent']:.1f}%")
print(f"   Auto: {len(valid_auto_corrected)}/{len(summaries_data)} valides ({len(valid_auto_corrected)/len(summaries_data)*100:.1f}%) - Taux rejet: {stats_auto_corrected['rejection_rate_percent']:.1f}%")

# Amélioration
improvement_manual = len(valid_corrected) - 143
improvement_auto = len(valid_auto_corrected) - 99

print(f"\n🚀 AMÉLIORATIONS:")
print(f"   Manuel: +{improvement_manual} résumés validés")
print(f"   Auto: +{improvement_auto} résumés validés")
print(f"   Réduction taux rejet manuel: {61.6 - stats_corrected['rejection_rate_percent']:.1f} points")
print(f"   Réduction taux rejet auto: {73.4 - stats_auto_corrected['rejection_rate_percent']:.1f} points")

# Validation des objectifs
target_met = 10 <= stats_corrected['rejection_rate_percent'] <= 20
print(f"\n✅ Objectif taux rejet 10-20%: {'ATTEINT' if target_met else 'À AJUSTER'}")
print(f"✅ Performance <50ms: {'ATTEINT' if stats_corrected['avg_processing_time_ms'] < 50 else 'À OPTIMISER'}")

In [ ]:
# Analyse par grades (vérification de la cohérence)
results_df = pd.DataFrame([
    {
        'grade': summaries_data[i]['quality_grade'],
        'valid_corrected': results_corrected[i].is_valid,
        'reasons': '; '.join(results_corrected[i].rejection_reasons) if results_corrected[i].rejection_reasons else 'Valid'
    }
    for i in range(len(summaries_data))
])

print("🎯 Validation par grade (après corrections):")
for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    if grade in results_df['grade'].values:
        grade_data = results_df[results_df['grade'] == grade]
        rejected = len(grade_data[~grade_data['valid_corrected']])
        total = len(grade_data)
        print(f"   Grade {grade}: {rejected}/{total} rejetés ({rejected/total*100:.1f}%)")

# Vérification cohérence: les grades A+ ne doivent pas être plus rejetés que les grades D
a_plus_rejection = len(results_df[(results_df['grade'] == 'A+') & (~results_df['valid_corrected'])]) / len(results_df[results_df['grade'] == 'A+']) * 100
d_rejection = len(results_df[(results_df['grade'] == 'D') & (~results_df['valid_corrected'])]) / len(results_df[results_df['grade'] == 'D']) * 100

coherence_ok = a_plus_rejection < d_rejection
print(f"\n✅ Cohérence grades (A+ < D): {'OK' if coherence_ok else 'À AJUSTER'} ({a_plus_rejection:.1f}% vs {d_rejection:.1f}%)")

In [ ]:
# Conclusion et validation
print("\n🎉 CONCLUSION DES CORRECTIONS:")

corrections_successful = (
    stats_corrected['rejection_rate_percent'] < 30 and  # Taux rejet raisonnable
    len(valid_corrected) > 200 and  # Assez de résumés valides
    stats_corrected['avg_processing_time_ms'] < 50 and  # Performance OK
    coherence_ok  # Cohérence par grades
)

if corrections_successful:
    print("✅ CORRECTIONS RÉUSSIES - Niveau 0 opérationnel")
    print("🚀 PRÊT pour le Niveau 1 : Détection heuristique")
else:
    print("⚠️ CORRECTIONS PARTIELLES - Ajustements supplémentaires nécessaires")
    
    if stats_corrected['rejection_rate_percent'] >= 30:
        print("   - Taux rejet encore trop élevé")
    if len(valid_corrected) <= 200:
        print("   - Pas assez de résumés valides")
    if not coherence_ok:
        print("   - Incohérence dans les grades")

print(f"\nMétriques finales:")
print(f"- Taux rejet: {stats_corrected['rejection_rate_percent']:.1f}% (objectif: 10-20%)")
print(f"- Résumés valides: {len(valid_corrected)}/372 ({len(valid_corrected)/372*100:.1f}%)")
print(f"- Performance: {stats_corrected['avg_processing_time_ms']:.1f}ms (objectif: <50ms)")